# Notebook explanation

The purpose of this notebook is to use a prompt given to some LLM ("LLM A") to generate a dataset of different prompts to be fed to some other LLM ("LLM B" - potentially the same as "LLM A") for which you have access to the weights. This dataset of prompts will be used to create a "representation vector" of a property or concept for LLM B in order to "steer" LLM B to have more of that property or concept in its outputs. This Notebook does not cover the creation of representations or steering, only the the creation of a dataset of prompts.

The kinds of prompts you want to generate should be about the property or concept you want to steer the LLM towards, not necessary literally mentioning it - e.g. prompts about politeness do not necessarily need to have the word polite in them. Part of the point of dataset creation is to explore which kinds of generated prompts yield good representations.

The Notebook works as follows:

- The Setup section loads Python libraries needed to run the code. You do not need to change anything here.
- The Inputs section is where you define the prompt you will use to generate your dataset of prompts. Instructions on how to do this are given. This is the only section of the notebook where you will need to change anything.
- The Review section then generates a small example dataset of prompts and shows them to you. If you like them, continue on to the end of the Notebook.
- If you do not like them, please go back to the Inputs section to refine your prompt to generate your dataset of prompts.
- The Dataset Generation section completes the dataset generation
- The View Dataset section loads your generated dataset for inspection.
- Your dataset will be stored in /data/inputs/name_of_your_dataset/dataset if you want to use it later.

The datasets will be generated in CSV format and should have the following form, where the first line is the column headings, the the next two lines are example prompts. The columns ethical_area and ethical_valence are two different labels (classifications) of the prompt. The Notebook will help you generate these labels. This example is for "politeness" prompts:

```
prompt, ethical_area, ethical_valence
"Would you be so kind as to pass the water please.", "polite", 1
"Give me the water now.", "impolite", 0
```

If this is too restrictive, you will be able to create other columns and the notebook will ask you about that too.

Note that where we have text, such as for the prompt or ethical_area, we want it enclosed in quote marks. Lone numbers do not need this.

If there is something unusual you want to do that the current notebook does not permist, please ask, or feel free to try adding code for it. 

# Setup (just run)

In [1]:
# Imports
import pandas as pd
import main
from omegaconf import DictConfig, OmegaConf
import yaml
from hydra import initialize
from hydra.core.global_hydra import GlobalHydra
from hydra.experimental import compose
import ipywidgets as widgets
from IPython.display import display

# For refactored code
# Need to tidy this up and remove duplicates

from data_handler import DataHandler
from data_analyser import DataAnalyzer
from model_handler import ModelHandler

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import FeatureAgglomeration

# For datsaet generation
import IPython
import json
import csv
import os
from jinja2 import Environment, FileSystemLoader
import math
import time
import os
import re

import yaml
from ipywidgets import widgets, VBox, Button, Checkbox, Text, IntText, FloatText, SelectMultiple, Label
from openai import OpenAI


/home/aayushkucheria/Documents/Steering-LLMs/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/aayushkucheria/Documents/Steering-LLMs/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:740: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


# Inputs

In [2]:
# Load existing template or create new one (menu/picker)
# Be able to load existing template - see the result on a form - be able to modify and save it
# Be able to create dictionary from form of placeholders and their content
# Need to be able to select number of new placeholders if creating from scratch or modifying


# Templates for labelling (menu/picker)

# Templates folder, labelling folder

# random seed but record seeds
# length
# Sample from top 5?

# higher shot in k-shot
# bigger model - more expensive

# What is the current context window?

Everything you might need to alter to change your prompt dataset generation requirements is in this inputs section of the Notebook.

!!!CAUTION!!! Enter below your OpenAI API key within the quote marks " ".

- This is not safe practice but will allow the Notebook to run.
- Better practice is to store the key in your environment variables. Please ask if you would like help with this.
- If you plan to push or share this code in any other way, make sure to remove your API key from this section.

```
client = OpenAI(
    api_key="your_api_key_goes_here"
)
```

In [3]:
#client = OpenAI(
#    api_key="fake_key"
#)

The code in the next cell represents the better practice for using your API key if it is saved in your environment variables.

- This code is currenty "commented out" (has the # symbol in front of each line). If your API key is saved in your environment variables and you want to use this code instead of the code in the previous cell, you will need to remove the # symbol from each line to make it work.
- There will be nothing to add to this code beyond removing the # symbols.
 

In [4]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

Enter below the name of the OpenAI model to use within the quote marks.

- Select the model from the list of allowable models in the OpenAI API given your subscription level (create an account and get an API ket here: https://auth0.openai.com/u/signup/identifier?state=hKFo2SA1REJ3dGFZVTllNHFvYUFkY2RrWEJpUUVMUWxvel91VqFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIFg1Z2NKOU9hUk4yYUFmWGxyTHlscmtNTmMxbDF5dWZTo2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q).
- You will need to put some token amount of money on to use gpt-4-0125-preview. It's something like $1. Might be $10.
- This is unlikely to change unless the list of available models is updated. See here for the list of models: https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo

In [ ]:
model = "gpt-4-0125-preview"

Enter below the model temperature. 

- From the OpenAI API documentation (https://platform.openai.com/docs/guides/text-generation/how-should-i-set-the-temperature-parameter): "Lower values for temperature result in more consistent outputs (e.g. 0.2), while higher values generate more diverse and creative results (e.g. 1.0). Select a temperature value based on the desired trade-off between coherence and creativity for your specific application. The temperature can range is from 0 to 2."
- If your generated sentences are too similar/boring, try increasing the number. Go the other way if too wacky.

In [ ]:
temperature = 1

Enter below the filename (the vaiable is called filestem) without the file extension (e.g. .csv) to save the dataset to.

- Don't worry about the file ending. It will be a csv. If you call your filename "honesty", then your dataset will be in honesty.csv
- Try to give it a specific name e.g. "honesty_v2.csv" or "honesty_pairs_v2.csv", something that will help you remember what it is especially if you are experimenting with variations.
- Giving it the same name as an existing dataset will currently overwrite the existing dataset. It should always end in ".csv"

In [ ]:
filestem = "honesty"

Enter below the total number of prompts you want to generate in your dataset of prompts.

- We are about to use a prompt to generate a datset of prompt examples.
- If you want to generate 10 prompts in total, put 10 here.
- If you want to generate 1000 prompts in total, put 1000 here.


In [ ]:
total_num_examples = 10

Enter below the number of examples per request. This is the number of prompt examples you want to generate in one request to the LLM.

- This is different from the total_num_examples variable you entered above.
- The reason we have this is because the LLM has a limit on the number of tokens it can process in one request. If you put a number that is too high here, you will not get the number of prompts you want.
- The number of tokens for gpt-4-0125-preview is 4096.
- From a quick internet search, I think this model uses about 1.3 tokens per word. So you might have about 4096/1.3 = 3150 words to play with if using this model. Remember, the number of tokens or words includes those in the prompt.
- Depending on the size of your prompt and the size of each example you plan to generate, you may need to adjust this number.
- This Notebook will make sure that the total number of examples you want to generate as defined in total_num_examples are generated, but it will make multiple requests to the LLM. Eg if you made total_num_examples = 100 and num_examples_per_request = 5, then this code will automatically make 100/5 = 20 requests to the LLM to generate the 100 examples you want.


In [5]:
num_examples_per_request = 5

## Notes for Aayush:

- The code below basically works. It's a first pass I did super fast with the new Claude 3 model (really good).
- I didn't try to understand it all or optimize it, I just wanted it to basically do what I wanted. sYou can see form looking at it is it not perfect and doesnlt completely make sense, but we can improve it.

## End of note for Aayush

If you want to modify an existing templete or create a new one, please run the cell below, otherwise skip to the next cell to load an existing template without modifying it.

To modify an existing template:
- You will be asked to load a select a template.
- Clicking on the template will load it into the Modify Template box below.
- Create a new filename and click save.
- You cannot overwirte existing templates. Modifications have to be saved with new names.

To create a new template (this could be better):
- Load any existing template
- Delete its contents
- Write what you want
- Create a new filename and click save.
- You cannot overwirte existing templates. New templates have to be saved with new names.

What is a template?
- A generic prompt stucture.
- Any variable/placeholder you might want to pass to your template should be include within a pair of braces like so: {{ your_variable_name }}


In [5]:
import os
from ipywidgets import Dropdown, Textarea, Button, VBox, Label, Text
from IPython.display import display
from jinja2 import Environment, FileSystemLoader


# Set up Jinja environment
template_dir = '../data/inputs/templates'
env = Environment(loader=FileSystemLoader(template_dir))

def load_templates():
    return [f for f in os.listdir(template_dir) if f.endswith(('.jinja', '.j2'))]

def create_template_dropdown():
    return Dropdown(options=load_templates(), description='Select Template:')

def create_template_textarea():
    return Textarea(description='Modify Template:', rows=10)

def create_filename_input():
    return Text(description='New Filename:', value='new_template.j2')

def create_save_button():
    button = Button(description='Save Template')
    button.on_click(save_modified_template)
    return button

def load_template_content(change):
    template_name = change['new']
    template_path = os.path.join(template_dir, template_name)
    try:
        with open(template_path, 'r') as f:
            template_content = f.read()
            template_textarea.value = template_content
    except IOError as e:
        print(f'Error loading template: {e}')

def save_modified_template(button):
    new_filename = new_filename_input.value
    _, extension = os.path.splitext(new_filename)
    if extension not in ['.jinja', '.j2']:
        print('Invalid file extension. Please use .jinja or .j2')
        return
    new_template_path = os.path.join(template_dir, new_filename)
    if os.path.exists(new_template_path):
        print(f'File "{new_filename}" already exists. Please choose a different filename.')
    else:
        modified_template = template_textarea.value
        try:
            with open(new_template_path, 'w') as f:
                f.write(modified_template)
                print(f'Template saved as "{new_filename}".')
        except IOError as e:
            print(f'Error saving template: {e}')

# Create widgets
template_dropdown = create_template_dropdown()
template_textarea = create_template_textarea()
new_filename_input = create_filename_input()
save_button = create_save_button()

# Set up event handler
template_dropdown.observe(load_template_content, names='value')

# Create layout
instructions_label = Label(value='Select a template to modify or create a new one:')
modify_label = Label(value='Modify the template:')
filename_label = Label(value='Enter a new filename (with .jinja or .j2 extension):')

layout = VBox([
    instructions_label,
    template_dropdown,
    modify_label,
    template_textarea,
    filename_label,
    new_filename_input,
    save_button
])

# Display the widgets
display(layout)

If you just want to work with an existing template and not load it, ignore the last cell and run this one.

- Select the template you want.
- You will be prompted to enter values for your variables/placeholders.
- You will also be asked to save this prompt (where templates and prompts are saved and how they can be related needs to 
be thought about).
- Create a new filename and click save.
- You cannot overwirte existing prompts. New prompts have to be saved with new names.

In [ ]:
import os
from ipywidgets import Dropdown, Text, Button, VBox, Output
from IPython.display import display, HTML
from jinja2 import Environment, FileSystemLoader, meta

# Set up Jinja environment
template_dir = '../data/inputs/templates'  # Directory where Jinja templates are stored
output_dir = '../data/inputs/prompts'  # Directory to save the rendered text
env = Environment(loader=FileSystemLoader(template_dir))

# Function to load template names from the directory
def load_templates():
    return [f for f in os.listdir(template_dir) if f.endswith(('.jinja', '.j2'))]

# Function to render the selected template with user input
def render_template(template_name, user_input):
    template = env.get_template(template_name)
    return template.render(**user_input)

# Create dropdown widget for template selection
template_dropdown = Dropdown(options=load_templates(), description='Select Template:')

# Create form for entering placeholder values
def create_load_form(template_name):
    template_source = env.loader.get_source(env, template_name)[0]
    parsed_content = env.parse(template_source)
    variables = meta.find_undeclared_variables(parsed_content)

    load_form = VBox()
    placeholders = {}

    for var in variables:
        placeholder_input = Text(description=f'Enter value for "{var}":')
        load_form.children += (placeholder_input,)
        placeholders[var] = placeholder_input

    output_filename_input = Text(description='Output Filename:', value='output.txt')
    load_form.children += (output_filename_input,)

    warning_output = Output()
    new_filename_input = Text(description='New Filename:')

    def save_template(button):
        user_input = {var: placeholder.value for var, placeholder in placeholders.items()}
        rendered_text = render_template(template_name, user_input)

        output_filename = output_filename_input.value
        output_path = os.path.join(output_dir, output_filename)

        if os.path.exists(output_path):
            warning_output.clear_output()
            with warning_output:
                print(f'File "{output_filename}" already exists. Enter a new filename.')
            new_filename_input.value = ''
            display(new_filename_input)
        else:
            with open(output_path, 'w') as f:
                f.write(rendered_text)
            print(f'Rendered text saved as "{output_filename}".')

    def save_with_new_filename(button):
        new_filename = new_filename_input.value
        output_path = os.path.join(output_dir, new_filename)
        with open(output_path, 'w') as f:
            f.write(rendered_text)
        print(f'Rendered text saved as "{new_filename}".')
        new_filename_input.value = ''

    save_button = Button(description='Save Template')
    save_button.on_click(save_template)
    load_form.children += (save_button,)

    new_filename_button = Button(description='Save with New Filename')
    new_filename_button.on_click(save_with_new_filename)

    load_form.children += (warning_output, new_filename_button)

    # Apply CSS styling to the form container
    load_form.layout.width = '100%'
    load_form.layout.min_width = '400px'
    load_form.add_class('my-form')

    return load_form

# Create button widget for loading the template form
def load_template_form(button):
    template_name = template_dropdown.value
    load_form = create_load_form(template_name)
    display(load_form)

load_form_button = Button(description='Load Template Form')
load_form_button.on_click(load_template_form)

# Define custom CSS styles
css_styles = '''
<style>
.my-form .widget-label {
    min-width: 300px;
    text-align: right;
    margin-right: 100px;
}
.my-form .widget-text {
    min-width: 500px;
}
</style>
'''

# Display the CSS styles
display(HTML(css_styles))

# Display the widgets
display(template_dropdown)
display(load_form_button)

Dropdown(description='Select Template:', options=('new_template_test.j2', 'aayush_testing_2.j2', 'new_template…

Button(description='Load Template Form', style=ButtonStyle())

In [6]:
import os
from ipywidgets import Dropdown, Textarea, Button, VBox, Label, Text, Output, HBox
from IPython.display import display, HTML
from jinja2 import Environment, FileSystemLoader, meta

In [11]:
# Set up Jinja environment
template_dir = '../data/inputs/templates'
output_dir = '../data/inputs/prompts'
env = Environment(loader=FileSystemLoader(template_dir))

def load_templates():
    return [f for f in os.listdir(template_dir) if f.endswith(('.jinja', '.j2'))]

def create_template_dropdown():
    return Dropdown(options=load_templates())

def create_template_content_input():
    return Textarea(rows=10,)

def create_filename_input():
    return Text(value='new_template.jinja')

def create_save_button():
    button = Button(description='Save Template')
    button.on_click(save_template)
    return button

def create_use_button():
    button = Button(description='Use Template')
    button.on_click(use_template)
    return button

def create_preview_button():
    button = Button(description='Preview Template')
    button.on_click(preview_template)
    return button

In [21]:
def save_template(button):
    new_filename = filename_input.value
    new_template_path = os.path.join(template_dir, new_filename)
    if os.path.exists(new_template_path):
        warning_output.clear_output()
        with warning_output:
            print(f'File "{new_filename}" already exists. Do you want to overwrite it?')
            overwrite_button = Button(description='Overwrite')
            overwrite_button.on_click(lambda _: save_template_content(new_template_path, True))
            cancel_button = Button(description='Cancel')
            cancel_button.on_click(lambda _: warning_output.clear_output())
            display(overwrite_button, cancel_button)
    else:
        save_template_content(new_template_path, False)

def save_template_content(file_path, overwrite=False):
    template_content = template_content_input.value
    with open(file_path, 'w') as f:
        f.write(template_content)
    success_output.clear_output()
    with success_output:
        print(f'Template saved as "{os.path.basename(file_path)}".')
    
    if not overwrite:
        # Refresh the template dropdown
        template_dropdown.options = load_templates()

def load_template_content(change):
    template_name = change['new']
    template_path = os.path.join(template_dir, template_name)
    with open(template_path, 'r') as f:
        template_content = f.read()
    template_content_input.value = template_content

def render_template(template_name, user_input):
    template = env.get_template(template_name)
    return template.render(**user_input)

def use_template(button):
    template_name = template_dropdown.value
    template_source = env.loader.get_source(env, template_name)[0]
    parsed_content = env.parse(template_source)
    variables = meta.find_undeclared_variables(parsed_content)

    load_form = VBox()
    placeholders = {}
    for var in variables:
        placeholder_input = Text(description=f'Enter value for "{var}":')
        load_form.children += (placeholder_input,)
        placeholders[var] = placeholder_input
        
    output_filename_input = Text(description='Enter output filename:')
    load_form.children += (output_filename_input,)

    # Apply CSS styling to the form container
    load_form.layout.width = '100%'
    load_form.layout.min_width = '400px'
    load_form.add_class('my-form')
    
    display(load_form)

    def render_and_save(button):
        placeholder_values = {var: widget.value for var, widget in placeholders.items()}
        rendered_text = render_template(template_name, placeholder_values)
        output_filename = output_filename_input.value
        output_path = os.path.join(output_dir, output_filename)
        
        if os.path.exists(output_path):
            print(f'File "{output_filename}" already exists.')
            return
        
        with open(output_path, 'w') as f:
            f.write(rendered_text)
            print(f'Rendered text saved as "{output_filename}".')
        
        # Update the preview output widget with the rendered template
        preview_output.clear_output()
        with preview_output:
            print('Template Preview:')
            print(rendered_text)

    render_button = Button(description='Save and Render')
    render_button.on_click(render_and_save)
    load_form.children += (render_button,)

In [22]:
def create_widgets():
    global template_dropdown, template_content_input, filename_input, save_button, use_button, preview_button, warning_output, success_output, preview_output
    template_dropdown = create_template_dropdown()
    template_content_input = create_template_content_input()
    filename_input = create_filename_input()
    save_button = create_save_button()
    use_button = create_use_button()
    # preview_button = create_preview_button()
    warning_output = Output()
    success_output = Output()
    preview_output = Output()

def create_layout():
    input_layout = VBox([
        Label(value='Template Manager', style={'font_weight': 'bold', 'font_size': '18px'}),    
        Label(value='Select a template to modify or create a new one:'),
        template_dropdown,
        Label(value='Edit the template content below:'),
        template_content_input,
        Label(value='Enter a filename to save the modified template (e.g. my_template.jinja or my_template.j2):'),
        filename_input,
    ])
    button_layout = HBox([
        save_button,
        use_button,
    ])
    output_layout = VBox([
        warning_output,
        success_output,
        # preview_output,
    ])
    main_layout = VBox([input_layout, button_layout, output_layout, preview_output])
    return main_layout

def initialize():
    create_widgets()
    template_dropdown.observe(load_template_content, names='value')
    layout = create_layout()
    display(layout)

In [23]:
initialize()

Rendered text saved as "aayush_testing_preview_correct_maybe".
File "aayush_testing_preview_correct_maybe" already exists.


## Notes for Aayush

- I've not updated the rest of the notebook since our meeting, and it was complete anyway. I'll leave more notes about what needs to be done.
- Two good things would be to (1) allow the user to read through and check the completed prompt; and (2) have the prompt read to send to the LLM
- We need to create a form for the labelling.

## End of notes for Aayush

Enter below your prompt for generating a dataset prompts. Things to note:
- You need to break up your prompt within the square brackets for readability. You simply write as much as you want on one line within quote marks " " and then add a comma at the end of the line. Then you can start a new line. The Notebook will reconstruct the prompt as one long string for you, putting spaces between the content of the different lines you have written.
- If you want to use quote marks in your prompt, you need to use inverted commas insted. Eg. Use 'this', not "this".
- You need to add the num_examples_per_request in the weird Python format as shown in the default example below. The curly brackets go where you want the number to go, then at the end of the line with the curly brackets, you put .format(num_examples_per_request) as in:


```
"Could you please write {} questions about honesty with exactly".format(num_examples_per_request)
```

- You only need to write enough in your prompt to generate new prompt examples. We are not trying to get it to classify the prompt as well at this stage, so you don't need to ask it to the classify the prompt as good or bad or anthying like that at this point. We will get to that.
- What we want is to create a single column CSV where the single column is a column of our generated prompt examples.
- We therefore need to tell the LLM about the structure of the output we want to make this easier. Complicated prompts such as prompts generating contrasting pairs (e.g. a good and bad version of the same prompt), will require more specific instructions as you can see in the default example below, because we want each prompt of the pair of contrasting prompts on seperate rows of the CSV.
- Whatever you do for 4, I recommend you always include the following at the end of your prompt:

```
"Do not write any other text in your output except for a correctly formatted CSV",
"such that your output text could be loaded as a valid CSV."
```

- Something to be aware of is that since we are asking the same prompt many times, e.g. we ask for 100 examples in total, 5 at a time, so we pass the same prompt to generate examples 100/5=20 times, the outputs may be a bit "samey" every five lines. You probably want to ask for as many examples per request as you can (e.g. 100 all at once if possible, rather than 5 at a time). Higher temeprature might help. We could also play with some other API parameters to address this. Let me know if you encounter such problems.

In [ ]:
prompt = [
        "Could you please write {} questions about honesty with exactly".format(num_examples_per_request),
        "the same structure as in my examaple? The questions should be",
        "asked exactly the same way twice, differing only in the ending.",
        "The ending is either 'The answer is A because' or 'The answer is B because'",
        "Here is my example:",
        "How should you approach honesty in your daily life?",
        "Select from the following options:",
        "A. Always be truthful and transparent in your interactions.",
        "B. Definitely by lying and deceiving whenever it benefits you!",
        "The answer is A because",
        "How should you approach honesty in your daily life?",
        "Select from the following options:",
        "A. Always be truthful and transparent in your interactions.",
        "B. Definitely by lying and deceiving whenever it benefits you!",
        "The answer is B because",
        "End of example.",
        "Format your output as a single column CSV.",
        "Each row of the CSV is a single question in quote marks followed by a comma,",
        "so each question in a pair of identical questions differing only in the ending is on a seperate row.",
        "Do not write any other text in your output except for a correctly formatted CSV",
        "such that your output text could be loaded as a valid CSV."
        ]

Enter below your prompts for generating dataset labels.

- These prompts will be fed to the model along with the full dataset of generated prompts after generation.
- You may need to experiment a little to get good labelling from the LLM.
- It is not certain that the LLm will be able to perform the labelling you are asking for, and if it can it may make mistakes.
- You need to check the labels are correct both in the Review section and after the full Dataset Generation section.
- Remember you need to generate labels for ethical_area and ethical_valency at a minimum.
- In the example below, ethical_area always map to the same ethical_valency and vice versa. In other words they are synomyms. They need not be though.
- Your label's name and the prompt to generate it should be enclosed in quote marks. 
- Where we want text based labels, e.g. "good" or "bad", please ask for them with quote marks around them. However, given the way this code is constructed, you need to ask for \"good\" or \"bad\", not "good" or "bad" - that is, add a backslash \ before each quote mark. This is because you are already typing within a pair of quote marks. 
- When asking for a number, do not ask for quote marks around them.
- You will see there is a sample new label called "your_new_label" that is commented out with the # symbol. For each new label of your own, just keep adding then like this with the label name in quote marks followed by a colon and then the prompt instruction in quote marks followed by a comma, but remove the # symbol at the start.

In [ ]:
labelling_questions = {
    "ethical_area": "Do you think the start of the response in the following text is good or bad? Output only the word \"Good\" for good, or \"Bad\" for bad in single word response within single quote marks. Here is the text: ",
    "ethical_valance": "Do you think the start of the response in the following text is good or bad? Output only the number 1 for good, or 0 for bad. Here is the text: ",
    # "your_new_label": 'On a scale of 0 to 10, how cool is this text? Coolness should be judged based on the style in the text, where 0 represents purely functional text with no style, and 10 represents style on a level of the best writers from history. Output only a number from 0 to 10. Here is the text: ",
}

In [ ]:
# Examples recreating various prompt datasets as seen in the literature.

# Review

You do not need to alter the code in any of the cells from here or on. What you do need to do is as follows: after running the code to the end of this section, just before the Dataset Generation Section, check that the kinds of prompts that are being generated for the new dataset of prompts are as you expected, and that any labelling of the data that is being done looks correct. This is to save you wasting time and money by avoiding generating the whole dataset but then not liking the results.

### !!! CHECK YOUR DATASET SAMPLE HERE BEFORE CONTINUING!!!

If you are happy with the results, continue, if not, go back to the start of the Inputs section and work through to this point again.

## Notes for Aayush

- I want to put all the code for this in a class called "data_creator"
- Nothing currently happens in the Review, Dataset Generation or View Dataset sections. The code for generating the dataset is all currently below these sections oin the Define Paths section which is from the old notebook. Basically lots of reorganizing and refactoring to be done.

## End of note for Aayush

In [ ]:
# 

# Dataset Generation

# View Dataset

In [ ]:
# Load full dataset

In [ ]:
# Look at random sample of dataset - useful for getting a sense if the data has been labelled correctly.

## Define paths

In [ ]:
dataset_dir = os.path.join("../data/inputs", filestem)
 
# Create the directory if it doesn't exist
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    print(f"Directory created: {dataset_dir}")
else:
    print(f"Directory already exists: {dataset_dir}")

In [ ]:
# Generate the dataset by calling the OpenAI API
def generate_dataset_from_prompt(prompt,
                                 generated_dataset_file_path,
                                 model,
                                 log_file_path,
                                 i):
    completion = client.chat.completions.create(
            **{
                "model": model,
                "temperature": temperature,
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            }
        )
    
    completion_words = completion.choices[0].message.content.strip()

    # cleaned_completion = completion.choices[0].message.content.strip()[3:-3]
    print(" ")
    print(completion_words)
    print(" ")

    # Open a file in write mode ('w') and save the CSV data
    with open(generated_dataset_file_path+"_"+str(i)+".txt", 'w', newline='', encoding='utf-8') as file:
        file.write(completion_words)

    num_words_in_prompt = count_words_in_string(prompt)
    num_words_in_completion = count_words_in_string(completion_words)
    total_words = num_words_in_prompt + num_words_in_completion

    num_tokens_in_prompt = completion.usage.prompt_tokens
    num_tokens_in_completion = completion.usage.completion_tokens
    total_tokens = num_tokens_in_prompt + num_tokens_in_completion

    prompt_cost = num_tokens_in_prompt*0.01/1000
    completion_cost = num_tokens_in_completion*0.03/1000
    total_cost = prompt_cost + completion_cost
    
    tokens_per_prompt_word = num_words_in_prompt/num_tokens_in_prompt
    tokens_per_completion_word = num_words_in_completion/num_tokens_in_completion

    log = {
            "num_words_in_prompt": num_words_in_prompt,
            "num_words_in_completion": num_words_in_completion,
            "total_words": total_words,
            "num_tokens_in_prompt": num_tokens_in_prompt,
            "num_tokens_in_completion": num_tokens_in_completion,
            "total_tokens": total_tokens,
            "prompt_cost": prompt_cost,
            "completion_cost": completion_cost,
            "total_cost": total_cost,
            "tokens_per_prompt_word": tokens_per_prompt_word,
            "tokens_per_completion_word": tokens_per_completion_word

    }

    for k, v in log.items():
        print(k, v)
    print(" ")

    with open(log_file_path+"_"+str(i)+".txt", 'w') as file:
        file.write(json.dumps(log, indent=4))

def count_words_in_string(input_string):
    words = input_string.split()
    return len(words)

In [ ]:
# Generate the dataset

# Convert the prompt from a list to a string
prompt = " ".join(prompt)

# First save the prompt as a text file
with open(dataset_dir+"/prompt.txt", 'w', newline='', encoding='utf-8') as file:
    file.write(prompt)

# Define the file path for the generated dataset
generated_dataset_file_path = os.path.join(dataset_dir, f"{filestem}")

# Define the log file path
log_file_path = os.path.join(dataset_dir, "log")

# Define the number of iterations
num_iterations = math.ceil(total_num_examples/num_examples_per_request)

start_time = time.time()

# Generate the dataset
for i in range(num_iterations):
    print("Iteration: ", i)
    generate_dataset_from_prompt(prompt, generated_dataset_file_path, model, log_file_path, i)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"The code took {elapsed_time} seconds to run.")

## Combine datasets

In [ ]:
# Get a list of all the files you want to process
# Makes sure they all have the same prompt context
# eg won;t mix up honest with justice etc
files = [os.path.join(generated_dataset_dir, f) for f in os.listdir(generated_dataset_dir) if f.endswith('.txt') and prompt_context in f]

# Define the regular expression pattern
# Get lines that start with a quote,
# then have any number of characters,
# then end with a quote and possibly comma
# We're trying to find all valid CSV lines
pattern = r'^\".*\",?[\r\n]*'

# Open the master CSV file
with open(combined_dataset_file_path, "a") as master:
    # Loop over the files
    for file in files:
        # Open the current file and read its contents
        with open(file, 'r') as f:
            content = f.read()

        # Use the re.findall function to find all matches in the content
        matches = re.findall(pattern, content, re.MULTILINE)        

        # Loop over the matches
        for match in matches:
            
            # Remove any trailing commas and newline characters
            match_cleaned = match.rstrip(',\r\n')
            
            # Append the match to the master CSV file
            master.write(match_cleaned + '\n')

## Add optional columns for classification

The columns added here work with the defaults currently hardcoded into the data analysis
This hardcoding will be resolved soon for more flexibility.

I show examples of using am LLM to auto-label and a way to label if you knw in advance which rows are of which kind of label.

### Example of labelling using an LLM

In [ ]:
def ask_openai(prompt):
    completion = client.chat.completions.create(
                **{
                    "model": model,
                    "messages": [
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt}
                    ]
                }
            )
    return completion.choices[0].message.content.strip()

input_file_path = combined_dataset_file_path
output_file_path = os.path.join(generated_dataset_dir, prompt_context+"_combined_dataset_ethical_area.csv")


with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile, \
     open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # Add a header
    writer.writerow(["Prompt", "Ethical Area"])
    
    for row in reader:
        # Assuming each row contains a single column with your text
        question = row[0]  # Adjust this if your structure is different
        # Here you define the question you want to ask about each row
        prompt = f"Do you think the start of the response in '{question}' is good or bad? Output only the word \"Good\" for good, or \"Bad\" for bad in single word response within single quote marks."
        response = ask_openai(prompt)
        # Add the OpenAI response to the row
        row.append(response)
        writer.writerow(row)

### Example of labelling programatically without LLM

In [ ]:
# Note we might not need to query the API for kind of lebelling
# Eg if we know the questions always go good, bad, good, bad, good etc

input_file_path = os.path.join(generated_dataset_dir, prompt_context+"_combined_dataset_ethical_area.csv")
output_file_path = os.path.join(generated_dataset_dir, prompt_context+"_combined_dataset_fully_labelled.csv")

with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile, \
     open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    # If your CSV has a header and you want to keep it, read and write it first
    # This also allows you to add a new column name to the header
    header = next(reader)
    header.append("Positive")  # Add your new column name here
    writer.writerow(header)
    
    # Enumerate adds a counter to an iterable and returns it (the enumerate object).
    for index, row in enumerate(reader, start=1):  # Start counting from 1
        if index % 2 == 0:  # Check if the row number is even
            row.append(0)
        else:
            row.append(1)
        writer.writerow(row)